In [1]:


############################################     Necessary Imports     ############################################

from Sub_Window_Download_Resize import *
from Sub_Window_Ml_Model import *
from Sub_Window_Prediction_WebCam import *

import tkinter as tk
from tkinter import *

from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk# , NavigationToolbar2TkAgg
import cv2
import os
import requests
import urllib.request as url
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from bs4 import BeautifulSoup
import shutil
import time
import concurrent.futures
import random
from PIL import ImageTk, Image

import tensorflow as tf
from tensorflow.keras import datasets, layers, models, losses, Model
from sklearn.model_selection import train_test_split



In [2]:
############################################      Saving the First Current Working Directory    ############################################

if ('is_dir_saved_once' in globals()) == False:
    is_dir_saved_once = False

if is_dir_saved_once == False:
    directory_needed_at_start = os.getcwd()
    is_dir_saved_once = True

os.chdir(directory_needed_at_start)

############################################      Creating Main GUI Window     ############################################

class Main_Home_Window:
    
    
    
    def __init__(self,  window_store_data):
        self.window_store_data = window_store_data
        
        self.window_store_data.attributes('-fullscreen', True)
        self.window_store_data.title('Data Entry')
        # Bind the ESC key with the callback function
        self.window_store_data.bind('<Escape>', lambda e: self.close_win(e, self.window_store_data))
        self.HEIGHT, self.WIDTH = 700, 1300
        self.website_name = ""
        self.create_main_window()
        
    def close_win(self, e, window):
        window.destroy()
        
    ###############################     Creating Canvas, Frames, Labels, Entries and Buttons   ###############################

    def create_main_window(self):
        
        canvas = tk.Canvas(self.window_store_data, height=self.HEIGHT, width=self.WIDTH)
        canvas.pack()

        frame_1 = tk.Frame(self.window_store_data, bg='#80c1ff', bd=5)
        frame_1.place(relx=0.05, rely=0.05, relwidth=0.9, relheight=0.3)

        label = tk.Label(frame_1, text="From which Website do you want to download the Images?", font=40)
        label.place(relx=0.05, rely=0.05, relwidth=0.9, relheight=0.4)

    

        r=StringVar()
        r.set("Unsplash")
        self.website_name = "Unsplash"
        button_1 = Radiobutton(frame_1, text="Google", font=40, variable=r, value="Google", command=lambda: self.save_website_name(r.get()))
        button_1.place(relx=0.05, rely=0.55, relwidth=0.47, relheight=0.4)

        button_2 = Radiobutton(frame_1, text="Unsplash", font=40, variable=r, value="Unsplash", command=lambda: self.save_website_name(r.get()))
        button_2.place(relx=0.55, rely=0.55, relwidth=0.4, relheight=0.4)

        frame_2= tk.Frame(self.window_store_data, bg='#80c1ff', bd=5)
        frame_2.place(relx=0.05, rely=0.40, relwidth=0.9, relheight=0.55)

        label = tk.Label(frame_2, text="Enter number of classes:-", font=40)
        label.place(relx=0.05, rely=0.05, relwidth=0.47, relheight=0.25)

        entry_no_classes = tk.Entry(frame_2, font=40)
        entry_no_classes.place(relx=0.55, rely=0.05, relwidth=0.40, relheight=0.25)

        entry_img_per_class = tk.Entry(frame_2, font=40)
        entry_img_per_class.place(relx=0.55, rely=0.35, relwidth=0.40, relheight=0.25)

        label = tk.Label(frame_2, text="Enter number of Images you want to train on per class:-", font=40)
        label.place(relx=0.05, rely=0.35, relwidth=0.47, relheight=0.25)
        
#         label = tk.Label(frame_2, 
#                          text="Achtung!! If you want to download Images from Google, do not set Images > 60!!"
#                         , font=1)
#         label.place(relx=0, rely=0.7, relwidth=0.6, relheight=0.25)

        button_3 = tk.Button(frame_2, text="Store Data", font=40, command=lambda: self.save_data(entry_no_classes.get(), entry_img_per_class.get()))
        button_3.place(relx=0.05, rely=0.65, relwidth=0.9, relheight=0.30)
    
    ###############################     Creating Command Methods   ###############################
        
    def save_website_name(self, value):
#         global website_name
        if value == 'Google':
            tk.messagebox.showinfo("Warning!!!","Keep Images per class to download less than 65")
        self.website_name = value
    #     print('Downloading images from {}...'.format(website_name))
    
    def save_data(self, x, y):
        
#         global no_classes, images_per_class
        self.no_classes, self.images_per_class = int(x), int(y)
        window_Download_and_Resize = Tk()
        window_Download_and_Resize.attributes('-fullscreen', True)
        window_Download_and_Resize.title('Download Images')
        
        # Bind the ESC key with the callback function
        window_Download_and_Resize.bind('<Escape>', lambda e: self.close_win(e, window_Download_and_Resize))
        
        self.window_store_data.destroy()
        New_Window_In_App = Download_Resize_Window(
            self.website_name, window_Download_and_Resize, self.HEIGHT, self.WIDTH, self.no_classes, self.images_per_class)
        window_Download_and_Resize.mainloop()
#         ask_question_for_classes(window_Download_and_Resize, HEIGHT, WIDTH, no_classes)
    
    ###############################     Creating Image Viewer Window       ###############################
    
    def view_images_and_labels(self, window, images_array, flag_top_level, classes, labels, no_classes, images_per_class):
    #         global window_images_and_labels
    
#         self.image_list = image_list
#         self.classes = classes
#         self.labels = labels
        if flag_top_level:
            window_images_and_labels = Toplevel(window)
        else:
            window_images_and_labels = Tk()
            
        window_images_and_labels.title('View Images')
        window_images_and_labels.grab_set()
        
        # Bind the ESC key with the callback function
        window_images_and_labels.bind('<Escape>', lambda e: Main_Home_Window.close_win(self, e, window_images_and_labels))
    #         window_images_and_labels.geometry("500x500")
#         global path, image_list
    
        image_list = []
    
        for idx, img in enumerate(self.X_train):
                image_list.append(ImageTk.PhotoImage(Image.fromarray((img * 255).astype(np.uint8)).resize((480, 480))))


        self.my_label = tk.Label(window_images_and_labels, image=image_list[0])
        self.my_label.grid(row=0, column=0, columnspan=3)

        self.label_class = tk.Label(window_images_and_labels, text=classes[int(labels[0])])
        self.label_class.grid(row=1, column=0, columnspan=3)
        
        def forward(image_number):
#                 global my_label, label_class
#                 global button_forward
#                 global button_back

            self.my_label.grid_forget()
            self.label_class.grid_forget()
            self.my_label = tk.Label(window_images_and_labels, image=image_list[image_number-1])
            self.label_class = tk.Label(window_images_and_labels, text=classes[int(labels[image_number-1])])

            self.button_forward = Button(window_images_and_labels, text=">>", command=lambda: forward(image_number+1))
            self.button_back = Button(window_images_and_labels, text="<<", command=lambda: back(image_number-1))

            if image_number == len(images_array):
                self.button_forward = Button(window_images_and_labels, text=">>", state=DISABLED)

            self.my_label.grid(row=0, column=0, columnspan=3)
            self.label_class.grid(row=1, column=0, columnspan=3)
            self.button_back.grid(row=2, column=0)
            self.button_forward.grid(row=2, column=2)

        def back(image_number):
#                 global my_label, label_class
#                 global button_forward
#                 global button_back

            self.my_label.grid_forget()
            self.label_class.grid_forget()
            self.my_label = Label(window_images_and_labels, image=image_list[image_number-1])
            self.label_class = tk.Label(window_images_and_labels, text=classes[int(labels[image_number-1])])

            self.button_forward = Button(window_images_and_labels, text=">>", command=lambda: forward(image_number+1))
            self.button_back = Button(window_images_and_labels, text="<<", command=lambda: back(image_number-1))

            if image_number == 1:
                button_back = Button(window_images_and_labels, text="<<", state=DISABLED)

            self.my_label.grid(row=0, column=0, columnspan=3)
            self.label_class.grid(row=1, column=0, columnspan=3)
            self.button_back.grid(row=2, column=0)
            self.button_forward.grid(row=2, column=2)

        def end_program():
#             os.chdir('..')
            window_images_and_labels.destroy()

        self.button_back = Button(window_images_and_labels, text="<<", command=back, state=DISABLED)
        self.button_exit = Button(window_images_and_labels, text="Exit Program", command=end_program)
        self.button_forward = Button(window_images_and_labels, text=">>", command=lambda: forward(2))


        self.button_back.grid(row=2, column=0)
        self.button_exit.grid(row=2, column=1)
        self.button_forward.grid(row=2, column=2)

        window_images_and_labels.mainloop()

In [3]:

window_store_data = tk.Tk()
App_creation = Main_Home_Window(window_store_data)
window_store_data.mainloop()